In [1]:
# requirements
!pip install -q rdflib sparqlwrapper owlrl gdown pandas wikipedia-api 

In [2]:
import gdown
import os
import pandas as pd
import rdflib
import urllib
from SPARQLWrapper import SPARQLWrapper
from urllib.parse import quote
from datetime import datetime
from dateutil import parser
import wikipediaapi
import owlrl

In [3]:
from rdflib import Graph, URIRef, BNode, Literal, FOAF, RDF, RDFS, XSD, Namespace
from SPARQLWrapper import JSON
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [4]:
!pip freeze > requirements.txt

In [5]:
# create directory for data
if not os.path.exists("./data/"):
    os.mkdir("./data/")

In [6]:
# Download data
gdown.download(id="1UILSMenvqzFMsIhV6l4ZV1opJpRQZnwC", output="./data/rym_top_5000_all_time.csv", quiet=True)
gdown.download(id="1UNJ2iDX-xoWIekSA0ZwoqNOk3eAZjqrp", output="./data/albumlist.csv", quiet=True)

'./data/albumlist.csv'

# **KRR Module 2 2022/23 Project**

The project consists in the creation of an RDF knowledge graph starting from tabular data (two CSV files loaded below).

You should cover all of the following steps.

### **`1. Create an RDFLib Graph from data contained in CSV files`**

In steps:
1) Load the CSV files in two pandas DataFrames and join them
2) Create an RDFLib `Graph` and add triples from the data of the dataframes

Useful links:

- [Pandas user guide](https://pandas.pydata.org/docs/user_guide/index.html) and [documentation](https://pandas.pydata.org/docs/reference/), e.g.:
    - [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html)
    - [Join dataframes](https://pandas.pydata.org/docs/user_guide/10min.html#join)
    - [Splitting and replacing strings](https://pandas.pydata.org/docs/user_guide/text.html#splitting-and-replacing-strings)
    - [Apply a function over an axis of a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html?highlight=apply#pandas.DataFrame.apply)

In [7]:
df1 = pd.read_csv('data/albumlist.csv', encoding='mac_roman')
df1.head()

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [8]:
df2 = pd.read_csv('data/rym_top_5000_all_time.csv')
df2.head()

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379


In [9]:
df1 = df1.drop('Number', axis=1)
df1.head()

,Year,Album,Artist,Genre,Subgenre
0,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [10]:
df2 = df2.drop('Ranking', axis=1)

In [11]:
df2 = df2.rename(columns={'Artist Name':'Artist'})

In [12]:
dfo = pd.merge(df1, df2, on=['Album', 'Artist'], how='outer')
print(dfo.shape)
dfo.head()

(5205, 11)


,Year,Album,Artist,Genre,Subgenre,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1967.0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock",1 June 1967,"Psychedelic Pop, Pop Rock","psychedelic, playful, melodic, male vocals, wa...",4.13,"43,576",863.0
1,1966.0,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock",16 May 1966,Baroque Pop,"Wall of Sound, warm, bittersweet, love, romant...",4.18,"36,305",727.0
2,1966.0,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock",5 August 1966,"Pop Rock, Psychedelic Pop","psychedelic, melodic, male vocals, drugs, ecle...",4.23,"43,178",1160.0
3,1965.0,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock",30 August 1965,"Folk Rock, Singer/Songwriter","poetic, cryptic, surreal, sarcastic, male voca...",4.17,"26,338",604.0
4,1965.0,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock,3 December 1965,Pop Rock,"melodic, love, male vocals, bittersweet, roman...",4.03,"33,508",721.0


In [13]:
g = Graph()
DBP = Namespace("http://dbpedia.org/property/")
EX = Namespace("http://www.example.org/")
g.bind("dbp", DBP)
g.bind("ex", EX)

In [14]:
DBO = Namespace("http://dbpedia.org/ontology/")

In [15]:
def encode_uri(uri):
    return URIRef(uri.replace(" ", "_").replace('"', ""))

Create an RDFLib `Graph` and add triples from the data of the dataframes**

For example, add triples like the following (but not only):

- `[album] dbp:artist [artist]`,
- `[album] rdfs:label [album_name]`,
- `[album] rdf:type [genre]`

Make sure you use URIs and Literals correctly.

In [16]:
for _, row in dfo.iterrows():
        album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
        
        if not pd.isna(row['Genre']):
            genres = row['Genre'].split(',')
            for genre in genres:
                subgenres = genre.split('/')
                for subgenre in subgenres:
                    subgenre = subgenre.replace("&", "").strip()
                    genre_uri = encode_uri("http://dbpedia.org/resource/" + subgenre + "_music")
                    g.add((album_uri, DBP.genre, genre_uri))


In [17]:
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    
    if not pd.isna(row['Subgenre']):
        subgenres = row['Subgenre'].split(',')
        for subgenre in subgenres:
            subgenre = subgenre.replace("&", "and").strip()
            subgenre_uri = encode_uri("http://dbpedia.org/resource/" + subgenre)
            g.add((album_uri, DBP.subgenre, subgenre_uri))


In [18]:
all_genres = set()

for _, row in dfo.iterrows():
    if pd.notna(row['Genre']):
        genres = [genre.strip() for genre in row['Genre'].replace(" / ", ",").split(',')]
        if "& Country" in genres:
            genres.remove("& Country")
            genres.append("Country")
        all_genres.update(genres)

for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    # We'll add genres and subgenres from the 'Genres' column
    if pd.notna(row['Genres']):
        genres_and_subgenres = [gs.strip() for gs in row['Genres'].replace(" / ", ",").split(',')]

        for gs in genres_and_subgenres:
            gs = gs.replace(",", "").strip()  # handle comma-separated genres
            if gs in all_genres:
                genre_uri = encode_uri("http://dbpedia.org/resource/" + gs + "_music")
                g.add((album_uri, DBP.genre, genre_uri))
            else:
                subgenre_uri = encode_uri("http://dbpedia.org/resource/" + gs)
                g.add((album_uri, DBP.subgenre, subgenre_uri))


In [19]:
# [album] dbp:artist [artist]
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    artist_uri = encode_uri("http://dbpedia.org/resource/" + row['Artist'])

    g.add((album_uri, DBP.artist, artist_uri))

In [20]:
# [album] rdfs:label [album_name]
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])

    g.add((album_uri, RDFS.label, Literal(row['Album'])))

In [21]:
# [album] dbp:year [Year]
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    year = row['Year']
    
    if not pd.isna(year):
        year_literal = Literal(int(year), datatype=XSD.gYear)
        g.add((album_uri, DBP.year, year_literal))

In [22]:
# [album] dbp:releaseDate [Release Date]
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    date = row['Release Date']
    
    if not pd.isna(date):
            release_date = parser.parse(date).date()
            date_literal = Literal(release_date.isoformat(), datatype=XSD.date)
            g.add((album_uri, DBP.releaseDate, date_literal))


In [23]:
# [album] dbp:averageRating [Average Rating]
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    
    if not pd.isna(row['Average Rating']):
        rating_literal = Literal(row['Average Rating'], datatype=XSD.float)
        g.add((album_uri, DBP.rating, rating_literal))

In [24]:
# [album] dbp:numberOfRatings [Number of Ratings]
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    
    if not pd.isna(row['Number of Ratings']):
        rating_str = row['Number of Ratings'].replace(",", "")
        rating_value = int(rating_str)
        rating_literal = Literal(rating_value, datatype=XSD.integer)
        g.add((album_uri, DBP.numberOfRatings, rating_literal))


In [25]:
# [album] dbp:numberOfReviews [Number of Reviews]
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    
    if not pd.isna(row['Number of Reviews']):
        reviews_value = int(rating_str)
        reviews_literal = Literal(reviews_value, datatype=XSD.integer)
        g.add((album_uri, DBP.numberOfReviews, reviews_literal))

In [26]:
for _, row in dfo.iterrows():
    album_uri = encode_uri("http://dbpedia.org/resource/" + row['Album'])
    
    if not pd.isna(row['Descriptors']):
        a = row['Descriptors'].split(',')
        for i in a:
            i = i.replace(' ', '')
            g.add((album_uri, EX.descriptors, Literal(i)))

Some entities appear as both album and artist or as both album and gebre/subgenre. This will be a problem for later inferences, so we need to disambiguate.

In [27]:
def disambiguate_albums_and_artists(graph):
    new_graph = Graph()

    album_entities = set()
    artist_entities = set()

    # Collect the entities that are used as albums and as artists
    for s, p, o in graph:
        if p == DBP.artist:
            album_entities.add(s)
            artist_entities.add(o)

    # Entities that are used both as an album and an artist
    conflicting_entities = album_entities.intersection(artist_entities)

    for s, p, o in graph:
        # If the subject is a conflicting entity, create a new album entity
        if s in conflicting_entities:
            s = URIRef(str(s) + "_(album)")
        new_graph.add((s, p, o))

    return new_graph

# Usage:
g = disambiguate_albums_and_artists(g)


In [28]:
def disambiguate_albums_and_genres(graph):

    albums = set()
    genres = set()
    
    for s, p, o in graph:
        if p == DBP.artist:
            albums.add(s)
        elif p == DBP.genre or p == DBP.subgenre:
            genres.add(o)
            
    overlap = albums.intersection(genres)
    
    # Create a new graph with updated subjects
    new_graph = Graph()
    
    for s, p, o in graph:
        if s in overlap:
            new_s = URIRef(str(s) + "_(album)")
        else:
            new_s = s
        new_graph.add((new_s, p, o))
    
    return new_graph

g = disambiguate_albums_and_genres(g)


In [29]:
g.serialize(destination='musicgraph1.ttl', format='turtle')

<Graph identifier=N86a50f6213b949f58004a3eda0c944cf (<class 'rdflib.graph.Graph'>)>

### **`2. Integrate your data with DBpedia's`**
                                                                                                                                                               
You MUST extend the knowledge you already have on your local KG with the following from DBpedia:     
- band members
- city of birth of members
- country of birth of members
- founding city of the band
- founding country of the band     

You can also add more information regarding albums (optional)



Recommendations and hints: 
- How to find IRIs from DBpedia that match the entities in the local KG?
  1) guessing the IRI with simple tricks:
     - the album 'Rubber Soul' on DBpedia is `dbr:Rubber_Soul`, and this 'translation' may work for other albums (don't do it manually) 
     - the album 'Led Zeppelin' on DBpedia is `dbr:Led_Zeppelin_(album)` as the same name is also associated to the band
  2) through their `rdfs:label`: for instance, the artist [Amy Winehouse](https://en.wikipedia.org/wiki/Amy_Winehouse) has 'Amy Winehouse' as label on DBpedia and therfore after the matching the IRI of the entity can be extracted
  3) using the [DBpedia lookup endpoint](https://github.com/dbpedia/lookup). The following request tries to find the DBpedia entity for the artist [Fabrizio De Andrè](https://en.wikipedia.org/wiki/Fabrizio_De_Andr%C3%A9): https://lookup.dbpedia.org/api/search/KeywordSearch?QueryString=fabrizio%20de%20andre&format=json&MaxHits=5
- restrict the search to entities of type 'Album', 'Band', 'Country', etc, in order to avoid overloading DBPedia's endpoint (see [here](http://mappings.dbpedia.org/server/ontology/classes/) for a list of DBPedia classes) and thus avoid high latency or being banned      
- note that sometimes useful properties to reach entities have the `dbp` prefix which is different from `dbo` ([more info](https://parklize.blogspot.com/2016/05/dbpedia-difference-between-dbo-and-dbp.html)). It's ok to use both
- here is a **non-exaustive** list of useful properties: `dbp:hometown`, `dbo:birthPlace`, `dbo:country`, `dbo:formerBandMember`
- how to find properties that link two entities of our interest?
  1) by checking the [DBpedia ontology](https://akswnc7.informatik.uni-leipzig.de/dstreitmatter/archivo/dbpedia.org/ontology/2023.05.12-020000/ontology_type=parsed.owl) manually
  2) by checking the DBpedia ontology with Protégé
  3) using the DBpedia resource pages and exploiting your knowledge (or Wikipedia information). For instance, let us consider [Metallica](https://en.wikipedia.org/wiki/Metallica), I know that [Dave Mustaine](https://en.wikipedia.org/wiki/Dave_Mustaine) was a member, therefore I ask myself: How are these two entities linked in the DBpedia KG? On the [DBpedia page of Metallica](https://dbpedia.org/page/Metallica) we can find that the property used is `dbo:formerBandMember` thus I can use this property to get band members 
  4) running SPARQL queries like the following in the DBpedia endpoint:
      ```
      SELECT DISTINCT ?p
      WHERE {
        ?s a dbo:Band .
        ?o a dbo:MusicalArtist .
        ?s ?p ?o .
      }
      ```
  5) if you are curious enough you can check [ABSTAT](http://abstat.disco.unimib.it/about.html), a tool for KG exploration with a nice interface
- please notice that DBpedia is not perfect so is reccomendable to check the type of the candidates before including them

Create a set of all the artists so I do not have to run multiple queries on the same artist.

In [30]:
unique_artists_complete = dfo['Artist'].unique()

Check if the artist is a Band, but if it is not try adding _(band) at the end of the URI and run the query again.

In [31]:
for artist in unique_artists_complete:
    artist_uri = encode_uri("http://dbpedia.org/resource/" + artist)

    # Construct SPARQL query
    query = f"""
    ASK {{
        <{artist_uri}> rdf:type dbo:Band .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    if results['boolean']:
        print(f"{artist_uri} is a band")
        g.add((URIRef(artist_uri), RDF.type, DBO.Band))
    else:

        # Add _band to the URI and check again
        artist_uri_band = artist_uri + "_(band)"
        query = f"""
        ASK {{
            <{artist_uri_band}> rdf:type dbo:Band .
        }}
        """
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        
        if results['boolean']:
            print(f"{artist_uri_band} is a band")
            g.add((URIRef(artist_uri_band), RDF.type, DBO.Band))
        else:
            print(f"{artist_uri_band} does not exist as a band in DBpedia")


For all the URIs where it was necessary to add _(band) at the end of the URI in order to make DBPedia find the
correct one, replace all the occurrences of the nonband URIs with the _(band) URI.

In [32]:
def replace_band_uris(graph):
    new_graph = Graph()

    band_uris = [str(s) for s, p, o in graph if str(s).endswith('_(band)')]
    non_band_uris = [uri.replace('_(band)', '') for uri in band_uris]

    for s, p, o in graph:
        
        if type(s) == URIRef and str(s) in non_band_uris:
            s = URIRef(str(s) + '_(band)')

        if type(o) == URIRef and str(o) in non_band_uris:
            o = URIRef(str(o) + '_(band)')

        new_graph.add((s, p, o))
    
    return new_graph

g = replace_band_uris(g)

In [33]:
def add_band_hometown(graph):
    new_graph = Graph()

    for s, p, o in graph:
        new_graph.add((s, p, o))

        # If the object is dbo:Band, query DBpedia for the band's hometown
        if o == DBO.Band:
            query = f"""
            SELECT ?hometown WHERE {{
                <{s}> dbo:hometown ?hometown .
                ?hometown a dbo:City .
            }}
            """
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)

            # Execute the query and get the results
            try:
                results = sparql.query().convert()
            except Exception as e:
                print(f"Failed to fetch hometown for band: {s}. Error: {e}")
                continue

            # Add each hometown to the graph
            for result in results["results"]["bindings"]:
                if "hometown" in result:
                    hometown = result["hometown"]["value"]
                    if hometown.startswith("http://dbpedia.org/resource/"):
                        hometown_uri = URIRef(hometown)
                        new_graph.add((s, DBO.hometown, hometown_uri))

    return new_graph

g = add_band_hometown(g)

In [34]:
def add_band_country_of_origin(graph):
    new_graph = Graph()

    # Iterate over all triples in the graph
    for s, p, o in graph:
        # Add the triple to the new graph
        new_graph.add((s, p, o))

        # If the object is dbo:Band, query DBpedia for the band's country of origin
        if o == DBO.Band:
            query = f"""
            SELECT ?country WHERE {{
                <{s}> dbo:hometown/dbo:country ?country .
                ?country a dbo:Country .
    
            }}
            """
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)

            # Execute the query and get the results
            try:
                results = sparql.query().convert()
            except Exception as e:
                print(f"Failed to fetch hometown and origin for band: {s}. Error: {e}")
                continue

            # Add each hometown and origin to the graph
            for result in results["results"]["bindings"]:
                if "country" in result:
                    country = result["country"]["value"]
                    if country.startswith("http://dbpedia.org/resource/"):
                        country_uri = URIRef(country)
                        new_graph.add((s, DBO.origin, country_uri))

    return new_graph

g = add_band_country_of_origin(g)


In [35]:
# Add current and former band members to the graph.
def add_band_members(graph):
    # Create a new graph to store the updated data
    new_graph = Graph()

    # Iterate over all triples in the graph
    for s, p, o in graph:
        # Add the triple to the new graph
        new_graph.add((s, p, o))

        # If the subject is a band, query DBpedia for its members
        if str(o) == str(DBO.Band):
            # Construct the SPARQL queries
            queries = [
                f"""
                SELECT ?member WHERE {{
                    <{s}> dbo:bandMember ?member .
                }}
                """,
                f"""
                SELECT ?member WHERE {{
                    <{s}> dbo:formerBandMember ?member .
                }}
                """
            ]

            for query in queries:
                sparql.setQuery(query)
                sparql.setReturnFormat(JSON)

                # Execute the query and get the results
                try:
                    results = sparql.query().convert()
                except:
                    print(f"Failed to fetch members for band: {s}")
                    continue

                # Determine whether the members are current or former members based on the query
                member_predicate = DBO.bandMember if 'dbo:bandMember' in query else DBO.formerBandMember

                # Add each band member to the graph
                for result in results["results"]["bindings"]:
                    member_uri = URIRef(result["member"]["value"])
                    new_graph.add((s, member_predicate, member_uri))

    return new_graph

g = add_band_members(g)


In [36]:
def add_birth_cities(graph):
    new_graph = Graph()

    # Iterate over all triples in the graph
    for s, p, o in graph:
        # Add the triple to the new graph
        new_graph.add((s, p, o))

        # If the predicate is bandMember, query DBpedia for the member's city of birth
        if str(p) == str(DBO.bandMember):
            # Construct the SPARQL query
            query = f"""
            SELECT ?city WHERE {{
                <{o}> dbo:birthPlace|dbp:birthPlace ?city .
                ?city a dbo:City . 
            }}
            """
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)

            # Execute the query and get the results
            try:
                results = sparql.query().convert()
            except:
                print(f"Failed to fetch city of birth for member: {o}")
                continue

            # Add each city of birth to the graph
            for result in results["results"]["bindings"]:
                city = result["city"]["value"]
                if city.startswith("http://dbpedia.org/resource/"):
                    city_uri = URIRef(city)
                    new_graph.add((o, DBO.birthPlace, city_uri))

    return new_graph

g = add_birth_cities(g)


In [37]:
def add_birth_country(graph):
    new_graph = Graph()

    # Iterate over all triples in the graph
    for s, p, o in graph:
        # Add the triple to the new graph
        new_graph.add((s, p, o))

        # If the predicate is bandMember, query DBpedia for the member's country of birth
        if str(p) == str(DBO.bandMember):
            query = f"""
            SELECT ?country WHERE {{
                <{o}> (dbo:birthPlace|dbp:birthPlace) ?place .
                OPTIONAL {{ ?place (dbo:country|dbp:country) ?country . }}
                FILTER EXISTS {{ ?country a dbo:Country }}.
            }}
            """
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)

            # Execute the query and get the results
            try:
                results = sparql.query().convert()
            except Exception as e:
                print(f"Failed to fetch country of birth for member: {o}. Error: {e}")
                continue

            # Add each country of birth to the graph
            for result in results["results"]["bindings"]:
                if "country" in result:
                    country = result["country"]["value"]
                    if country.startswith("http://dbpedia.org/resource/"):
                        country_uri = URIRef(country)
                        new_graph.add((o, DBO.country, country_uri))
 
    return new_graph

g = add_birth_country(g)

In [38]:
g.serialize(destination='musicgraph2.ttl', format='turtle')

# **`3. Gathering Information from unstructured data`**

Knowledge can be gathered from unstructured data as well as structured data. A great encyclopaedic source is Wikipedia whose articles follow a very strict pattern. For instance, the first lines about a person are reserved for biographical data and his or her occupation, therefore we can exploit it to understand the role of a band member and add this information to our KG.

In steps:
1) get the article from Wikipedia and extract the summary. You can use [Wikipedia-api](https://pypi.org/project/Wikipedia-API/) (see the code below)
2) extract the band member role (guitarist, bassist, keyboardist, singer, drummer, etc) using rules
3) add this information to the KG

Recommendations and hints: 
- some artists have multiple roles, add them all
- some artists have multiple bands, it is not required to understand the role for each band, keep it simple.
- extraction rules can be very simple (simple string matchers + if-else)  or more complex ones (e.g., regex, ChatGPT requests), it doesn't matter in terms of points.


In [39]:
g = Graph()
g.parse('musicgraph2.ttl', format = 'turtle')

In [40]:
band_member_roles = ["guitarist", "bassist", "keyboardist", "singer", "drummer", "vocalist", "songwriter", "lead vocalist", "lead singer" , "rhythm guitarist", "backing vocalist", "bass guitarist", "lead guitarist", "musician"]

In [41]:
def extract_band_member_roles(summary, band_member_roles):
    roles = []
    for role in band_member_roles:
        if role.lower() in summary.lower():
            roles.append(role)
    return roles


In [42]:
def add_band_member_roles(graph):
    new_graph = graph

    # Iterate over each triple in the graph
    for s, p, o in graph:
        if p == URIRef("http://dbpedia.org/ontology/bandMember") or p == URIRef("http://dbpedia.org/ontology/formerBandMember"):
            band_member_uri = str(o)
            band_member_title = urllib.parse.unquote(band_member_uri.split('/')[-1])

            # Get the Wikipedia page for the band member
            wiki_wiki = wikipediaapi.Wikipedia('en')
            page_py = wiki_wiki.page(band_member_title)

            if page_py.exists():
                print("Wikipedia page found for ", o)
                summary = page_py.summary[:200]
                roles = extract_band_member_roles(summary, band_member_roles)

                # Add band member roles to the knowledge graph
                for role in roles:
                    print(f"Adding role '{role}' for band member '{band_member_title}'")
                    new_graph.add((o, EX.role, Literal(role)))
            else:
                print("Wikipedia page not found for ", o)
    return new_graph


g = add_band_member_roles(g)

In [43]:
g.serialize(destination='musicgraph3.ttl', format='turtle')

### **`4. Include a small ontology and a  genres taxonomy`**

Using RDFlib, add classes such as Person, MusicAlbum, and so on to your KG. Add the domain and range for each property you use.

This will be needed later for inferring the types of the entities of your KG.

In [44]:
g = Graph()
g.parse("musicgraph3.ttl", format="turtle")

<Graph identifier=N81ddbbe745fd446c92dd5591a7e34c74 (<class 'rdflib.graph.Graph'>)>

In [45]:
OWL = Namespace("http://www.w3.org/2002/07/owl#")

In [46]:
# Create classes
g.add((DBO.Album, RDF.type, RDFS.Class))
g.add((DBO.Person, RDF.type, RDFS.Class))
g.add((DBO.MusicGenre, RDF.type, RDFS.Class))
g.add((DBO.MusicArtist, RDF.type, RDFS.Class))
g.add((DBO.BandMember, RDF.type, RDFS.Class))
g.add((DBO.City, RDF.type, RDFS.Class))
g.add((DBO.Country, RDF.type, RDFS.Class))
g.add((DBO.Band, RDF.type, RDFS.Class))

# Set domain and range for properties
g.add((DBO.country, RDFS.domain, DBO.BandMember))
g.add((DBO.country, RDFS.range, DBO.Country))

g.add((DBO.birthPlace, RDFS.domain, DBO.BandMember))
g.add((DBO.birthPlace, RDFS.range, DBO.City))

g.add((DBP.genre, RDFS.domain, DBO.Album))
g.add((DBP.genre, RDFS.range, DBO.MusicGenre))

g.add((DBP.subgenre, RDFS.domain, DBO.Album))
g.add((DBP.subgenre, RDFS.range, DBO.MusicGenre))

g.add((DBO.bandMember, RDFS.domain, DBO.Band))
g.add((DBO.bandMember, RDFS.range, DBO.BandMember))

g.add((DBO.formerBandMember, RDFS.domain, DBO.Band))
g.add((DBO.formerBandMember, RDFS.range, DBO.BandMember))

g.add((DBP.artist, RDFS.domain, DBO.Album))
g.add((DBP.artist, RDFS.range, DBO.MusicArtist))

g.add((DBO.origin, RDFS.domain, DBO.Band))
g.add((DBO.origin, RDFS.range, DBO.Country))

g.add((DBO.hometown, RDFS.domain, DBO.Band))
g.add((DBO.hometown, RDFS.range, DBO.City))

g.add((DBP.numberOfRatings, RDFS.domain, DBO.Album))
g.add((DBP.numberOfRatings, RDFS.range, XSD.integer))

g.add((DBP.numberOfReviews, RDFS.domain, DBO.Album))
g.add((DBP.numberOfReviews, RDFS.range, XSD.integer))

g.add((DBP.rating, RDFS.domain, DBO.Album))
g.add((DBP.rating, RDFS.range, XSD.float))

g.add((DBP.year, RDFS.domain, DBO.Album))
g.add((DBP.year, RDFS.range, XSD.gYear))

g.add((DBP.releaseDate, RDFS.domain, DBO.Album))
g.add((DBP.releaseDate, RDFS.range, XSD.date))

g.add((EX.descriptors, RDFS.domain, DBO.Album))
g.add((EX.descriptors, RDFS.range, XSD.string))

g.add((EX.role, RDFS.domain, DBO.BandMember))
g.add((EX.role, RDFS.range, XSD.string))

g.add((DBO.birthPlace, RDFS.domain, DBO.BandMember))
g.add((DBO.birthPlace, RDFS.range, DBO.City))

g.add((DBO.country, RDFS.domain, DBO.BandMember))
g.add((DBO.country, RDFS.range, DBO.Country))

g.add((EX.ratingCategory, RDFS.domain, DBO.Album))
g.add((EX.ratingCategory, RDFS.range, XSD.string))

g.add((DBP.bandGenres, RDFS.domain, DBO.Band))
g.add((DBP.bandGenres, RDFS.range, DBO.MusicGenre))

# Set subClassOf relations
g.add((DBO.Band, RDFS.subClassOf, DBO.MusicArtist))

g.add((DBO.BandMember, RDFS.subClassOf, DBO.Person))






<Graph identifier=N81ddbbe745fd446c92dd5591a7e34c74 (<class 'rdflib.graph.Graph'>)>

Moreover, the first dataframe has 'Genre' and 'Subgenre' columns.
Extract a genres taxonomy from the dataset and include it in your RDF KG. This will be needed later for inferring genres from subgenres.

E.g.: add triples to the graph like:

- `[subgenre] rdfs:subClassOf [genre]`.
- `[album] rdf:type [genre]`.

This should be done for all genres and subgenres (with an iteration, not manually).

Each word is mapped to its genre. This allows me to infer that rap is a subClassOf Hip Hop, which would not be possible with a simple string matching approach.
At this stage, I had to make some stylistic choices, such as mapping swing to Jazz and metal to Rock.

In [47]:
genre_mappings = {
    "rock": "Rock",
    "pop": "Pop",
    "funk": "Funk",
    "soul": "Soul",
    "hop": "Hip_Hop",
    "country": "Country",
    "jazz": "Jazz",
    "latin": "Latin",
    "blues": "Blues",
    "world": "World",
    "classical": "Classical",
    "metal": "Rock",
    "folk": "Folk",
    "reggae": "Reggae",
    "house": "Eleactronic",
    "electronic": "Electronic",
    "punk" : "Rock",
    "grunge" : "Rock",
    "rap" : "Hip_Hop",
    "bop" : "Jazz",
    "swing" : "Jazz"
}

In [48]:
def get_genre_subgenre_triples(graph, genre_mappings):
    new_triples = []
    
    for s, p, o in graph:
        if p == DBP.genre:
            new_triples.append((s, RDF.type, o))
        if p == DBP.subgenre:
            subgenre = str(o).split('/')[-1].replace("_", " ").lower()
            for genre_key, genre_value in genre_mappings.items():
                if genre_key in subgenre:
                    genre_uri = URIRef('http://dbpedia.org/resource/' + genre_value.replace(" ", "_") + '_music')
                    subgenre_uri = o
                    new_triples.append((subgenre_uri, RDFS.subClassOf, genre_uri))
                new_triples.append((s, RDF.type, o))
    
    return new_triples


In [49]:
new_triples = get_genre_subgenre_triples(g, genre_mappings)

for triple in new_triples:
    g.add(triple)


In [50]:
g.serialize("musicgraph4.ttl", format="turtle")

### **`5. Materialize inferences`**

Add to your KG inferences that you can make from the data on your KG (e.g., with SPARQL queries, or with OWLRL, ...).

**<ins>Some</ins>**  **<ins>examples</ins>**
- if you have that `[album] rdf:type [genre1]` and `[genre1] rdfs:subClassOf [genre2]`, add the triple `[album] rdf:type [genre2]`. Same for domains, ranges, subclasses, subproperties, OWL features, etc.
- if the nationality of a group member is missing, you can infer it from the nationality of the band

Run a few SPARQL queries on the graph to see the inferred triples.


In [51]:
g = Graph()
g = g.parse("musicgraph4.ttl", format = "turtle")

In [52]:
owlrl.DeductiveClosure(owlrl.RDFS_OWLRL_Semantics).expand(g)

As many bandMembers are missing a dbo:country property because it was not possible to retrieve their country of
origin for some of them, we will infer their dbo:country from the dbo:origin of their band.
We do that for both bandMembers and formerBandMembers.

In [53]:
for band in g.subjects(RDF.type, DBO.Band):
    band_country = g.value(band, DBO.origin)
    
    if band_country:
        for band_member in g.objects(band, DBO.bandMember):
            if not g.value(band_member, DBO.country):
                g.add((band_member, DBO.country, band_country))

In [54]:
for band in g.subjects(RDF.type, DBO.Band):
    band_country = g.value(band, DBO.origin)
    
    if band_country:
        for band_member in g.objects(band, DBO.formerBandMember):
            if not g.value(band_member, DBO.country):
                g.add((band_member, DBO.country, band_country))

We can add a scale where we can rate the Albums based on their rating between:

> 4-5 : Great

> 3-4 : Good

> 2-3 : Average

> 1-2 : Bad

> 0-1 : Horrible

In [55]:
for s in g.subjects(RDF.type, DBO.Album):
    rating = g.value(s, DBP.rating)
    if rating is not None:
        
        if 4 <= rating <= 5:
            rating_category = "Great"
        elif 3 <= rating < 4:
            rating_category = "Good"
        elif 2 <= rating < 3:
            rating_category = "Average"
        elif 1 <= rating < 2:
            rating_category = "Bad"
        else:
            rating_category = "Horrible"
        
        g.add((s, EX.ratingCategory, Literal(rating_category)))

I will add bandGenres triple for bands to later query for bands that have songs in different genres.
I will infer them from the albums they were dbp:artist of.

In [56]:
def update_graph_with_bandGenres(graph):

    query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbp: <http://dbpedia.org/property/>
    
    CONSTRUCT {
      ?band dbp:bandGenres ?genre .
    }
    WHERE {
      ?album dbp:artist ?band ;
              a ?genre .
      ?genre a dbo:MusicGenre .
      ?band a dbo:Band .
    }
    """
    
    new_graph = graph.query(query)

    # Merge the new triples with the existing graph
    graph += new_graph

    return graph


g = update_graph_with_bandGenres(g)


In [22]:
g.serialize("finalmusicgraph2.ttl", format="turtle")

<Graph identifier=Ndf9afdb6d5ae43b0aebcdcdb5b2070a4 (<class 'rdflib.graph.Graph'>)>

In [22]:
qres = g.query(
    """
    PREFIX ns1: <http://www.example.org/>
    SELECT ?album ?rating_category
    WHERE {
        ?album ns1:ratingCategory ?rating_category.
    }
    """)

for row in qres:
    print("%s rated as %s" % row)

http://dbpedia.org/resource/!!Destroy-Oh-Boy!! rated as Great
http://dbpedia.org/resource/#1_Record rated as Great
http://dbpedia.org/resource/'Allelujah!_Don't_Bend!_Ascend! rated as Great
http://dbpedia.org/resource/'Frisco_Mabel_Joy rated as Great
http://dbpedia.org/resource/'Round_About_Midnight rated as Great
http://dbpedia.org/resource/(A)_Senile_Animal rated as Great
http://dbpedia.org/resource/(I'm)_Stranded rated as Great
http://dbpedia.org/resource/(Listen_For)_The_Rag_and_Bone_Man rated as Great
http://dbpedia.org/resource/(What's_the_Story)_Morning_Glory? rated as Great
http://dbpedia.org/resource/(_) rated as Great
http://dbpedia.org/resource/(pronounced_'lĕh-'nérd_'skin-'nérd) rated as Great
http://dbpedia.org/resource/+'Justments rated as Great
http://dbpedia.org/resource/...And_Justice_for_All rated as Great
http://dbpedia.org/resource/...And_Out_Come_the_Wolves rated as Great
http://dbpedia.org/resource/...And_the_Battle_Begun rated as Great
http://dbpedia.org/resource

In [15]:
query = """
    SELECT ?s
    WHERE {
        ?s a rdfs:Class .
    }
"""

results = g.query(query)

for result in results:
    print(result)
    

(rdflib.term.URIRef('http://dbpedia.org/ontology/Album'),)
(rdflib.term.URIRef('http://dbpedia.org/ontology/Band'),)
(rdflib.term.URIRef('http://dbpedia.org/ontology/BandMember'),)
(rdflib.term.URIRef('http://dbpedia.org/ontology/City'),)
(rdflib.term.URIRef('http://dbpedia.org/ontology/Country'),)
(rdflib.term.URIRef('http://dbpedia.org/ontology/MusicArtist'),)
(rdflib.term.URIRef('http://dbpedia.org/ontology/MusicGenre'),)
(rdflib.term.URIRef('http://dbpedia.org/ontology/Person'),)


Check for MusicArtists that are also bands to check the range related inference.

In [16]:
query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?s
    WHERE {
        ?s a dbo:MusicArtist .
        ?s a dbo:Band .
    }
"""

results = g.query(query)

for result in results:
    print(result)



(rdflib.term.URIRef('http://dbpedia.org/resource/!T.O.O.H.!'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/...And_You_Will_Know_Us_by_the_Trail_of_Dead'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/12_Rods'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/13th_Floor_Elevators'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/16_Horsepower'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/ABBA'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/ABC_(band)'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/A_Sei_Voci'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/A_Silver_Mt._Zion'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Aaliyah'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Acrimony'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Afro-Cuban_All_Stars'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/After_Crying'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/After_the_Burial'),)
(rdflib.term.URIRef('http://dbpedia.org/resour

Check if I have any DBO.Person in the Graph to see if < Resource > a dbo:Person was inferred from 
the range of the bandMember property + dbo:BandMember rdfs:subClassOf dbo:Person

In [17]:
query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?s
    WHERE {
        ?s a dbo:Person .
    }
"""

results = g.query(query)

for result in results:
    print(result)


(rdflib.term.URIRef('http://dbpedia.org/resource/%22Sneaky%22_Pete_Kleinow'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/2-D_(character)'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/A-Plus_(rapper)'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/A._W._Yrjänä'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Aaron_Dalbec'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Aaron_Dessner'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Aaron_Harris_(drummer)'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Aaron_Lee_Tasjan'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Aaron_Rossi'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Aaron_Stainthorpe'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Aaron_Turner'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Abdul_%22Duke%22_Fakir'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Abe_Cunningham'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Abradab'),)
(rdflib.term.URIRef('http://dbp

Check for albums that are both Southern_Rock and Rock_music / Punk_Rock and Rock_music to see if the subclass relations added in the genres taxonomy section hold.

In [18]:
query = """

    SELECT ?s
    WHERE {
        ?s a <http://dbpedia.org/resource/Rock_music> .
        ?s a <http://dbpedia.org/resource/Southern_Rock> .
    }
"""

results = g.query(query)

for result in results:
    print(result)



(rdflib.term.URIRef("http://dbpedia.org/resource/(pronounced_'leh-'nerd_'skin-'nerd)"),)
(rdflib.term.URIRef("http://dbpedia.org/resource/(pronounced_'lĕh-'nérd_'skin-'nérd)"),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Beggars_Banquet'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Brothers_and_Sisters'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Decoration_Day'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Dixie_Chicken'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Dose'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Eat_a_Peach'),)
(rdflib.term.URIRef("http://dbpedia.org/resource/Feats_Don't_Fail_Me_Now"),)
(rdflib.term.URIRef("http://dbpedia.org/resource/Hittin'_the_Note"),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Idlewild_South'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/It_Still_Moves'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Life_Before_Insanity'),)
(rdflib.term.URIRef("http://dbpedia.org/resource/Sailin'_Shoes")

### **`6. Query the Graph`**

Query your graph in a way that:
- the usefulness of having added information from dbpedia is highlighted
- inference is exploited


**<ins>Some</ins>**  **<ins>examples</ins>** with the intentions:


| ID | Query | Intention/insight |
|----|------|-------------------|
| 1  | which bands have at least one member with a different nationality? | just curiosity |
| 2  | Which are the artist with many bands? | to spot session musicians, long-career artists, 'eclectic' artists |
| 3  | which are the countries with more [ Latin Rock ]( https://en.wikipedia.org/wiki/Latin_rock ) bands? and what about [ Black Metal ]( https://en.wikipedia.org/wiki/Black_metal )? | I suppose that latin rock is more popular in Latin America countries while Black Metal in Scandinavian countries, but I may be wrong! |
| 4  | Is there any artist who is a member of both a [ Britpop ]( https://en.wikipedia.org/wiki/Britpop ) band and a [ Hip hop ]( https://en.wikipedia.org/wiki/Hip_hop_music ) band | maybe to spot bands/artists with a strong musical contamination? e.g., [Damon Albarn ](https://en.wikipedia.org/wiki/Damon_Albarn) |
| 5  | Which is the city with more bands in absolute? and what if we consider only [ Grunge ]( https://en.wikipedia.org/wiki/Grunge )? | maybe to discover a music scene or just a very influential city. |
| 6  | Give me some band/artist from the same city of [ 2Pac ]( https://en.wikipedia.org/wiki/Tupac_Shakur ) | could be a way to find similar musical artists. What about adding also a time period and the musical genre? |
| 7  | which are the bands with Argentine and Spanish members? | maybe because I love both the Argentine and Spanish sound and I hope there is something that matches both? e.g., [Los Rodriguez](https://en.wikipedia.org/wiki/Los_Rodr%C3%ADguez) |




**Feel free to play and get cool insights**

Find Countries with black metal bands. 

In [8]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?country (COUNT(?band) as ?count) 
    WHERE {
        ?band a ns3:Band ;
            ns3:origin ?country ;
            ns1:bandGenres <http://dbpedia.org/resource/Black_Metal> .
    }
    GROUP BY ?country
    ORDER BY DESC(?count)
    """)

for row in qres:
    print("%s has %s Black Metal bands" % row)


http://dbpedia.org/resource/Norway has 6 Black Metal bands
http://dbpedia.org/resource/Poland has 3 Black Metal bands
http://dbpedia.org/resource/France has 2 Black Metal bands
http://dbpedia.org/resource/Finland has 2 Black Metal bands
http://dbpedia.org/resource/Switzerland has 1 Black Metal bands
http://dbpedia.org/resource/United_States has 1 Black Metal bands


Find the cities with the most Bands.

In [9]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?city (COUNT(?band) as ?count) 
    WHERE {
        ?band a ns3:Band ;
            ns3:hometown ?city .
    }
    GROUP BY ?city
    ORDER BY DESC(?count)
    """)

for row in qres:
    print("%s has %s bands" % row)


http://dbpedia.org/resource/London has 52 bands
http://dbpedia.org/resource/New_York_City has 28 bands
http://dbpedia.org/resource/Los_Angeles has 28 bands
http://dbpedia.org/resource/San_Francisco has 19 bands
http://dbpedia.org/resource/Seattle has 15 bands
http://dbpedia.org/resource/Birmingham has 14 bands
http://dbpedia.org/resource/Stockholm has 13 bands
http://dbpedia.org/resource/Long_Island has 10 bands
http://dbpedia.org/resource/Melbourne has 8 bands
http://dbpedia.org/resource/Chicago has 8 bands
http://dbpedia.org/resource/Helsinki has 8 bands
http://dbpedia.org/resource/Gothenburg has 8 bands
http://dbpedia.org/resource/Manchester has 8 bands
http://dbpedia.org/resource/Hamburg has 8 bands
http://dbpedia.org/resource/Sydney has 8 bands
http://dbpedia.org/resource/Boston has 8 bands
http://dbpedia.org/resource/Austin,_Texas has 7 bands
http://dbpedia.org/resource/Buenos_Aires has 7 bands
http://dbpedia.org/resource/San_Francisco,_California has 7 bands
http://dbpedia.org/r

Where is Jazz music the most popular?

In [10]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?city (COUNT(?band) as ?count) 
    WHERE {
        ?band a ns3:Band ;
            ns3:hometown ?city ;
            ns1:bandGenres <http://dbpedia.org/resource/Jazz_music> .
            
    }
    GROUP BY ?city
    ORDER BY DESC(?count)
    """)

for row in qres:
    print("%s has %s bands" % row)


http://dbpedia.org/resource/New_York_City has 5 bands
http://dbpedia.org/resource/London has 3 bands
http://dbpedia.org/resource/Los_Angeles has 3 bands
http://dbpedia.org/resource/Canterbury has 3 bands
http://dbpedia.org/resource/Buenos_Aires has 2 bands
http://dbpedia.org/resource/Chicago has 1 bands
http://dbpedia.org/resource/Phoenix,_Arizona has 1 bands
http://dbpedia.org/resource/Dakar has 1 bands
http://dbpedia.org/resource/Oxnard,_California has 1 bands
http://dbpedia.org/resource/East_Oakland,_Oakland,_California has 1 bands
http://dbpedia.org/resource/Long_Beach,_California has 1 bands
http://dbpedia.org/resource/Milan has 1 bands
http://dbpedia.org/resource/Mülheim_an_der_Ruhr has 1 bands
http://dbpedia.org/resource/Miami,_Florida has 1 bands
http://dbpedia.org/resource/Brooklyn,_New_York has 1 bands
http://dbpedia.org/resource/Munich has 1 bands
http://dbpedia.org/resource/Tokyo has 1 bands
http://dbpedia.org/resource/Cambridge,_Massachusetts has 1 bands
http://dbpedia.org

In [11]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?country (COUNT(?band) as ?count) 
    WHERE {
        ?band a ns3:Band ;
            ns3:origin ?country ;
            ns1:bandGenres <http://dbpedia.org/resource/Jazz_music> .
            
    }
    GROUP BY ?country
    ORDER BY DESC(?count)
    """)

for row in qres:
    print("%s has %s bands" % row)


http://dbpedia.org/resource/United_States has 18 bands
http://dbpedia.org/resource/United_Kingdom has 7 bands
http://dbpedia.org/resource/Kent has 3 bands
http://dbpedia.org/resource/Argentina has 2 bands
http://dbpedia.org/resource/Senegal has 1 bands
http://dbpedia.org/resource/Italy has 1 bands
http://dbpedia.org/resource/Japan has 1 bands
http://dbpedia.org/resource/Australia has 1 bands
http://dbpedia.org/resource/France has 1 bands


Which are the bands with Italian members?

In [12]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?band (COUNT(?bandMember) as ?count) 
    WHERE {
        ?band a ns3:Band ;
            ns3:bandMember ?bandMember .
        ?bandMember a ns3:BandMember ;
                ns3:country <http://dbpedia.org/resource/Italy> .
            
    }
    GROUP BY ?band
    ORDER BY DESC(?count)
    """)

for row in qres:
    print("%s has %s Italian BandMembers" % row)


http://dbpedia.org/resource/Premiata_Forneria_Marconi has 4 Italian BandMembers
http://dbpedia.org/resource/Le_Orme has 3 Italian BandMembers
http://dbpedia.org/resource/Elio_e_le_Storie_Tese has 2 Italian BandMembers
http://dbpedia.org/resource/Area_(band) has 1 Italian BandMembers
http://dbpedia.org/resource/Angra_(band) has 1 Italian BandMembers
http://dbpedia.org/resource/Museo_Rosenbach has 1 Italian BandMembers
http://dbpedia.org/resource/Sumo_(band) has 1 Italian BandMembers
http://dbpedia.org/resource/Whitesnake has 1 Italian BandMembers


Which are the Bands that have bandMembers from Argentina?

In [13]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?band (COUNT(?bandMember) as ?count) 
    WHERE {
        ?band a ns3:Band ;
            ns3:bandMember ?bandMember .
        ?bandMember a ns3:BandMember ;
                ns3:country <http://dbpedia.org/resource/Argentina> .
            
    }
    GROUP BY ?band
    ORDER BY DESC(?count)
    """)

for row in qres:
    print("%s has %s BandMembers from Argentina" % row)


http://dbpedia.org/resource/Sumo_(band) has 8 BandMembers from Argentina
http://dbpedia.org/resource/Serú_Girán has 4 BandMembers from Argentina
http://dbpedia.org/resource/Soda_Stereo has 2 BandMembers from Argentina
http://dbpedia.org/resource/Los_Fabulosos_Cadillacs has 2 BandMembers from Argentina
http://dbpedia.org/resource/Therion_(band) has 1 BandMembers from Argentina
http://dbpedia.org/resource/The_Mars_Volta has 1 BandMembers from Argentina
http://dbpedia.org/resource/Viper_(band) has 1 BandMembers from Argentina


And what about Spain?

In [14]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?band (COUNT(?bandMember) as ?count) 
    WHERE {
        ?band a ns3:Band ;
            ns3:bandMember ?bandMember .
        ?bandMember a ns3:BandMember ;
                ns3:country <http://dbpedia.org/resource/Spain> .
            
    }
    GROUP BY ?band
    ORDER BY DESC(?count)
    """)

for row in qres:
    print("%s has %s BandMembers from Spain" % row)


http://dbpedia.org/resource/Héroes_del_Silencio has 4 BandMembers from Spain
http://dbpedia.org/resource/Leño has 3 BandMembers from Spain
http://dbpedia.org/resource/Extremoduro has 2 BandMembers from Spain


Check the number of ratings to find the most popular albums and artists in a specific time. What about the decade 1980 - 1990?

In [15]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    SELECT ?album ?artist ?numberOfRatings
    WHERE {
        ?album a ns3:Album ;
            ns1:artist ?artist ;
            ns1:numberOfRatings ?numberOfRatings ;
            ns1:releaseDate ?releaseDate ;
            ns1:year ?year .
        FILTER ((?releaseDate >= "1980-01-01"^^xsd:date && ?releaseDate < "1991-01-01"^^xsd:date) 
        || (?year >= "1980"^^xsd:gYear && ?year < "1991"^^xsd:gYear))
    }
    ORDER BY DESC(?numberOfRatings)
    """)

for row in qres:
    print("%s by %s has %s ratings " % row)


http://dbpedia.org/resource/Remain_in_Light by http://dbpedia.org/resource/Talking_Heads has 36196 ratings 
http://dbpedia.org/resource/The_Queen_Is_Dead by http://dbpedia.org/resource/The_Smiths has 35400 ratings 
http://dbpedia.org/resource/Doolittle by http://dbpedia.org/resource/Pixies has 34101 ratings 
http://dbpedia.org/resource/Master_of_Puppets by http://dbpedia.org/resource/Metallica has 30780 ratings 
http://dbpedia.org/resource/Disintegration by http://dbpedia.org/resource/The_Cure has 29436 ratings 
http://dbpedia.org/resource/Closer by http://dbpedia.org/resource/Joy_Division has 28898 ratings 
http://dbpedia.org/resource/Daydream_Nation by http://dbpedia.org/resource/Sonic_Youth has 27963 ratings 
http://dbpedia.org/resource/Surfer_Rosa by http://dbpedia.org/resource/Pixies has 24269 ratings 
http://dbpedia.org/resource/Let_It_Be by http://dbpedia.org/resource/The_Beatles has 21363 ratings 
http://dbpedia.org/resource/Let_It_Be by http://dbpedia.org/resource/The_Beatles 

I am a huge U2 fan! Let's see which other bands are from Ireland as well.

In [16]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?band ?origin 
    WHERE {
        ?band a ns3:Band ;
            ns3:origin ?origin .
        <http://dbpedia.org/resource/U2> ns3:origin ?origin .
        
            
    }
    """)

for row in qres:
    print("%s is from %s like U2" % row)


http://dbpedia.org/resource/Altar_of_Plagues is from http://dbpedia.org/resource/Republic_of_Ireland like U2
http://dbpedia.org/resource/My_Bloody_Valentine_(band) is from http://dbpedia.org/resource/Republic_of_Ireland like U2
http://dbpedia.org/resource/Primordial_(band) is from http://dbpedia.org/resource/Republic_of_Ireland like U2
http://dbpedia.org/resource/U2 is from http://dbpedia.org/resource/Republic_of_Ireland like U2
http://dbpedia.org/resource/Thin_Lizzy is from http://dbpedia.org/resource/Republic_of_Ireland like U2


Are there any artists that played both in Black Metal bands and Pop bands?

In [20]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns2: <http://dbpedia.org/resource/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?artist ?band1 ?band2 
    WHERE {
        ?artist a ns3:BandMember .
        ?band1 a ns3:Band ;
               ns1:bandGenres <http://dbpedia.org/resource/Black_Metal> .
        OPTIONAL { ?band1 ns3:bandMember ?artist }
        OPTIONAL { ?band1 ns3:formerBandMember ?artist }
        ?band2 a ns3:Band ;
               ns1:bandGenres <http://dbpedia.org/resource/Pop_music> .
        OPTIONAL { ?band2 ns3:bandMember ?artist }
        OPTIONAL { ?band2 ns3:formerBandMember ?artist }
    } LIMIT 300
    """)
for row in qres:
    print("%s played in both bands: %s and %s" % row)


http://dbpedia.org/resource/%22Sneaky%22_Pete_Kleinow played in both bands: http://dbpedia.org/resource/Behemoth_(band) and http://dbpedia.org/resource/ABBA
http://dbpedia.org/resource/%22Sneaky%22_Pete_Kleinow played in both bands: http://dbpedia.org/resource/Behemoth_(band) and http://dbpedia.org/resource/ABC_(band)
http://dbpedia.org/resource/%22Sneaky%22_Pete_Kleinow played in both bands: http://dbpedia.org/resource/Behemoth_(band) and http://dbpedia.org/resource/Almendra_(band)
http://dbpedia.org/resource/%22Sneaky%22_Pete_Kleinow played in both bands: http://dbpedia.org/resource/Behemoth_(band) and http://dbpedia.org/resource/Alvvays
http://dbpedia.org/resource/%22Sneaky%22_Pete_Kleinow played in both bands: http://dbpedia.org/resource/Behemoth_(band) and http://dbpedia.org/resource/Antony_and_the_Johnsons
http://dbpedia.org/resource/%22Sneaky%22_Pete_Kleinow played in both bands: http://dbpedia.org/resource/Behemoth_(band) and http://dbpedia.org/resource/Bjork
http://dbpedia.org

Which bands have active members from Ireland and United Kindom?

In [28]:
qres = g.query(
    """
    PREFIX ns1: <http://dbpedia.org/property/>
    PREFIX ns3: <http://dbpedia.org/ontology/>
    SELECT ?artist1 ?artist2 ?band 
    WHERE {
        ?artist1 a ns3:BandMember ;
                ns3:country <http://dbpedia.org/resource/Republic_of_Ireland> .
        ?artist2 a ns3:BandMember ;
                ns3:country <http://dbpedia.org/resource/United_Kingdom> .
        ?band a ns3:Band ;
              ns3:bandMember ?artist1 ;
              ns3:bandMember ?artist2 .
    }
    """)

for row in qres:
    print("%s from Ireland and %s from United Kingdom play for: %s" % row)


http://dbpedia.org/resource/Bono from Ireland and http://dbpedia.org/resource/Adam_Clayton from United Kingdom play for: http://dbpedia.org/resource/U2
http://dbpedia.org/resource/Bono from Ireland and http://dbpedia.org/resource/The_Edge from United Kingdom play for: http://dbpedia.org/resource/U2
http://dbpedia.org/resource/John_Mitchell_(musician) from Ireland and http://dbpedia.org/resource/Mick_Pointer from United Kingdom play for: http://dbpedia.org/resource/Arena_(band)
http://dbpedia.org/resource/Larry_Mullen_Jr. from Ireland and http://dbpedia.org/resource/Adam_Clayton from United Kingdom play for: http://dbpedia.org/resource/U2
http://dbpedia.org/resource/Larry_Mullen_Jr. from Ireland and http://dbpedia.org/resource/The_Edge from United Kingdom play for: http://dbpedia.org/resource/U2
http://dbpedia.org/resource/Mark_Kelly_(keyboardist) from Ireland and http://dbpedia.org/resource/Ian_Mosley from United Kingdom play for: http://dbpedia.org/resource/Marillion
http://dbpedia.or

### **`Notes`**
- attempts to enrich the data (e.g., using Wikipedia articles, other KGs and data sources) will be positively evaluated, but only if enrichment in steps 2 and 3 has been sufficiently explored
- sometimes matching local entities with DBpedia entities is difficult: do not worry too much about that, we will give more importance to the choices and motivations rather than the end result of the matching
- as you can see, the instructions leave enough freedom, however in case of non-obvious clarifications or technical issues contact me at renzo.alvaprincipe@unimib.it or use the course forum. Questions about how to do the exercise will be ignored
- plagiarism will be **<ins>severely</ins>** punished for both parties